You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:
Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and What You Need to Know

Introduction:
- Brief overview of Artificial Intelligence (AI) and its impact on various industries
- Mention of the rapid advancements and innovations in AI technology
- Preview of key trends, players, and news in the AI space

Key Points:
1. Latest Trends in Artificial Intelligence:
- Deep learning and neural networks
- Natural language processing and conversational AI
- AI in healthcare, finance, and autonom

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Latest Trends, Key Players, and What You Need to Know

**Introduction:**
Artificial Intelligence (AI) has revolutionized various industries with its impact on automation, data analysis, and decision-making processes. The rapid advancements in AI technology have led to increased efficiency, improved accuracy, and innovative solutions across different sectors. In this blog post, we will explore the latest trends, key players, and noteworthy news in the AI space, providing valuable insights for tech enthusiasts, industry professionals, and individuals interested in the future of technology.

**Latest Trends in Artificial Intelligence:**
Deep learning and neural networks have been at the forefront of AI innovation, enabling machines to learn from data and make complex decisions. Natural language processing and conversational AI have also seen significant progress, with applications in virtual assistants, 

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and What You Need to Know

**Introduction:**
Artificial Intelligence (AI) has revolutionized various industries with its impact on automation, data analysis, and decision-making processes. The rapid advancements in AI technology have led to increased efficiency, improved accuracy, and innovative solutions across different sectors. In this blog post, we will explore the latest trends, key players, and noteworthy news in the AI space, providing valuable insights for tech enthusiasts, industry professionals, and individuals interested in the future of technology.

**Latest Trends in Artificial Intelligence:**
Deep learning and neural networks have been at the forefront of AI innovation, enabling machines to learn from data and make complex decisions. Natural language processing and conversational AI have also seen significant progress, with applications in virtual assistants, chatbots, and language translation services. AI is making waves in healthcare, finance, and autonomous vehicles, transforming the way we approach medical diagnosis, financial analysis, and transportation systems. Ethical considerations in AI development, such as bias in algorithms and data privacy, are crucial topics that need to be addressed as AI continues to evolve.

**Key Players in the AI Industry:**
Tech giants like Google, Microsoft, Amazon, and others are leading the way in AI research and development, investing heavily in AI technologies and applications. Startups and research institutions are also playing a vital role in driving AI innovation, with collaborations between academia and industry fostering groundbreaking discoveries. The impact of AI on the job market and workforce is a significant concern, as automation and AI technologies reshape traditional roles and create new opportunities for skilled workers.

**Noteworthy News in Artificial Intelligence:**
Recent breakthroughs in AI research, such as advancements in computer vision, reinforcement learning, and generative models, are pushing the boundaries of what AI can achieve. Applications of AI in solving complex problems, from climate change to cybersecurity, are demonstrating the potential of AI to address global challenges. Regulatory developments in AI governance, including guidelines for responsible AI use and accountability mechanisms, are shaping the future of AI policy and ethics. Public perception and awareness of AI technology are also evolving, with discussions on the benefits and risks of AI becoming more widespread.

**Call to Action:**
Stay informed about AI developments by following reputable news sources and research institutions to deepen your understanding of AI technology. Explore online courses and resources to enhance your knowledge of AI and machine learning, and share your thoughts and opinions on AI in the comments section or on social media platforms to engage in meaningful discussions about the future of AI.

In conclusion, Artificial Intelligence is a dynamic and transformative field with endless possibilities for innovation and growth. By staying informed and actively participating in the AI community, we can help shape the future of technology and harness the power of AI for the benefit of society.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Blog Article on "YOUR TOPIC HERE"

1. Latest Trends, Key Players, and Noteworthy News:
- Latest Trends: Discuss the emergence of new technologies in YOUR TOPIC HERE industry and how they are shaping the future.
- Key Players: Highlight the top companies leading the way in YOUR TOPIC HERE and their innovative approaches.
- Noteworthy News: Include recent developments, partnerships, or achievements within the YOUR TOPIC HERE sector.

2. Target Audience Analysis:
- Target Audience: Professionals workin

I now can give a great answer

Final Answer:
# Embracing Innovation: Exploring the Latest Trends in YOUR TOPIC HERE

In today's rapidly evolving landscape, the YOUR TOPIC HERE industry is experiencing a wave of new technologies that are reshaping the future. From cutting-edge innovations to disruptive solutions, the industry is witnessing a shift towards more efficient and sustainable practices. Companies are embracing these trends to stay ahead of the curve and meet the growing demands of consumers.

Leading the charge are top players in the YOUR TOPIC HERE sector, who are driving innovation and setting new standards for excellence. Their forward-thinking approaches and strategic partnerships are paving the way for a more dynamic and competitive industry. By highlighting their achievements and groundbreaking initiatives, we can gain valuable insights into the future direction of the YOUR TOPIC HERE landscape.

Recent developments and partnerships within the YOUR TOPIC HERE sector have

In [14]:
Markdown(result)

# Embracing Innovation: Exploring the Latest Trends in YOUR TOPIC HERE

In today's rapidly evolving landscape, the YOUR TOPIC HERE industry is experiencing a wave of new technologies that are reshaping the future. From cutting-edge innovations to disruptive solutions, the industry is witnessing a shift towards more efficient and sustainable practices. Companies are embracing these trends to stay ahead of the curve and meet the growing demands of consumers.

Leading the charge are top players in the YOUR TOPIC HERE sector, who are driving innovation and setting new standards for excellence. Their forward-thinking approaches and strategic partnerships are paving the way for a more dynamic and competitive industry. By highlighting their achievements and groundbreaking initiatives, we can gain valuable insights into the future direction of the YOUR TOPIC HERE landscape.

Recent developments and partnerships within the YOUR TOPIC HERE sector have also been a key focus, showcasing the industry's resilience and adaptability in the face of challenges. These collaborations and advancements are not only shaping the present but also laying the foundation for future growth and expansion. By staying informed about these noteworthy news, professionals can better navigate market changes and capitalize on emerging opportunities.

# Navigating the Landscape: Opportunities and Future Outlook in YOUR TOPIC HERE

As we look towards the future, the YOUR TOPIC HERE industry holds immense potential for growth and innovation. By analyzing current market trends and the strategies of key players, we can uncover valuable insights into the opportunities that lie ahead. From market insights to best practices, professionals can gain a competitive edge by staying informed and proactive in their approach.

In conclusion, the YOUR TOPIC HERE industry is on the cusp of a new era, driven by innovation, collaboration, and forward-thinking strategies. By keeping abreast of the latest trends and developments, professionals can position themselves for success and lead the way towards a brighter future. Subscribe for regular updates on YOUR TOPIC HERE trends and join the conversation with industry experts to stay ahead of the curve.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).